# HW02: Tokenization

Remember that these homework work as a completion grade. **You can skip one section without losing credit.**

In [1]:
#Import the AG news dataset (same as hw01)
#Download them from here 
!wget https://raw.githubusercontent.com/mhjabreel/CharCnn_Keras/master/data/ag_news_csv/train.csv

import pandas as pd
import nltk
df = pd.read_csv('train.csv')

df.columns = ["label", "title", "lead"]
label_map = {1:"world", 2:"sport", 3:"business", 4:"sci/tech"}
def replace_label(x):
	return label_map[x]
df["label"] = df["label"].apply(replace_label) 
df["text"] = df["title"] + " " + df["lead"]
df.head()

--2023-03-01 14:35:03--  https://raw.githubusercontent.com/mhjabreel/CharCnn_Keras/master/data/ag_news_csv/train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 29470338 (28M) [text/plain]
Saving to: ‘train.csv’

train.csv           100%[===================>]  28.10M  --.-KB/s    in 0.1s    

2023-03-01 14:35:04 (198 MB/s) - ‘train.csv’ saved [29470338/29470338]



,label,title,lead,text
0,business,Carlyle Looks Toward Commercial Aerospace (Reu...,Reuters - Private investment firm Carlyle Grou...,Carlyle Looks Toward Commercial Aerospace (Reu...
1,business,Oil and Economy Cloud Stocks' Outlook (Reuters),Reuters - Soaring crude prices plus worries\ab...,Oil and Economy Cloud Stocks' Outlook (Reuters...
2,business,Iraq Halts Oil Exports from Main Southern Pipe...,Reuters - Authorities have halted oil export\f...,Iraq Halts Oil Exports from Main Southern Pipe...
3,business,"Oil prices soar to all-time record, posing new...","AFP - Tearaway world oil prices, toppling reco...","Oil prices soar to all-time record, posing new..."
4,business,"Stocks End Up, But Near Year Lows (Reuters)",Reuters - Stocks ended slightly higher on Frid...,"Stocks End Up, But Near Year Lows (Reuters) Re..."


## Preprocess Text

In [35]:
import spacy
dfs = df.sample(50)
nlp = spacy.load('en_core_web_sm')

##TODO use spacy to split the documents in the sampled dataframe (dfs) in sentences and tokens

def split_doc(text):
    '''
    Returns -> tuple:
    1. list -> sentences
    2. list -> tokens
    '''
    doc = nlp(text)
    sentences = [sent for sent in doc.sents]
    tokens = [token for token in doc if not token.is_punct and not token.is_stop]

    return sentences, tokens

# '*' iterates over each single sample of dfs
dfs['sentences'], dfs['tokens'] = zip(*dfs['text'].apply(split_doc))

##TODO print the first sentence of the first document in your sample

print(dfs.iloc[0]['sentences'][0])

Tight US supplies boost oil price Oil prices have surged to two-week highs following news that supplies in the US are tightening.


In [36]:
dfs.head()

,label,title,lead,text,sentences,tokens
23541,business,Tight US supplies boost oil price,Oil prices have surged to two-week highs follo...,Tight US supplies boost oil price Oil prices h...,"[(Tight, US, supplies, boost, oil, price, Oil,...","[Tight, supplies, boost, oil, price, Oil, pric..."
86938,sport,Tressel: #39;Allegations Made Against Me Tota...,"COLUMBUS, Ohio -- Ohio State football coach Ji...",Tressel: #39;Allegations Made Against Me Tota...,"[(Tressel, :, , #, 39;Allegations, Made, Agai...","[Tressel, , 39;Allegations, Totally, False, 3..."
72971,sci/tech,Chip-Maker Agere Posts Loss on Charges (AP),"AP - Agere Systems Inc., which makes computer ...",Chip-Maker Agere Posts Loss on Charges (AP) AP...,"[(Chip, -, Maker, Agere, Posts, Loss, on, Char...","[Chip, Maker, Agere, Posts, Loss, Charges, AP,..."
92615,world,"Israelis, Palestinians Expect Rice to Implemen...",The Palestinian Authority is looking forward t...,"Israelis, Palestinians Expect Rice to Implemen...","[(Israelis, ,, Palestinians, Expect, Rice, to,...","[Israelis, Palestinians, Expect, Rice, Impleme..."
85740,sci/tech,"Europeans debate software patents, interoperab...",BRUSSELS -- The European Union's (E.U.'s) plan...,"Europeans debate software patents, interoperab...","[(Europeans, debate, software, patents, ,, int...","[Europeans, debate, software, patents, interop..."


In [45]:
##TODO create a new column with tokens in lowercase (x.lower()), without punctuation tokens (x.is_punct), stopwords (x.is_stop), and digits (x.is_digit)


def tokens_to_text(sample_of_tokens):
    tokens_text = [token.text.lower() for token in sample_of_tokens]

    return tokens_text

dfs['tokens_text'] = dfs['tokens'].apply(tokens_to_text)

##TODO print the tokens (x.lemma_) and the dependency labels (x.dep_ ) of the first sentence of the first document (doc.sents)

print(dfs.iloc[0]['sentences'][0])

for token in dfs.iloc[0]['sentences'][0]:
    print('token: {:15}, lemma: {:15}, dependency: {}'.format(token.text, token.lemma_, token.dep_))


Tight US supplies boost oil price Oil prices have surged to two-week highs following news that supplies in the US are tightening.
token: Tight          , lemma: tight          , dependency: compound
token: US             , lemma: US             , dependency: compound
token: supplies       , lemma: supply         , dependency: nsubj
token: boost          , lemma: boost          , dependency: ROOT
token: oil            , lemma: oil            , dependency: compound
token: price          , lemma: price          , dependency: compound
token: Oil            , lemma: oil            , dependency: compound
token: prices         , lemma: price          , dependency: nsubj
token: have           , lemma: have           , dependency: aux
token: surged         , lemma: surge          , dependency: ccomp
token: to             , lemma: to             , dependency: prep
token: two            , lemma: two            , dependency: nummod
token: -              , lemma: -              , dependency: punct


### Named Entities

Let's compute the ratio of named entities starting with a capital letter, e.g. if we have "University of Chicago" as a NE, "University" and "Chicago" are capitalized, "of" is not, thus the ratio is 2/3.

In [ ]:
##TODO print the ratio of tokens being part of a named entity span starting with a capital letter (doc.ents)

In [ ]:
##TODO print the ratio of capitalized tokens not being part of a named entity span (have no token.ent_type_)
# e.g. "The dog barks" = 1/3; 3 tokens, only "The" is capitalized

In [ ]:
##TODO print the ratio of capitalized tokens not being a named entity and not being the first token in a sentence
# e.g. "The dog barks" = 0; 3 tokens, "The" is capitalized but the starting token of a sentence, no other tokens are capitalized.

Give an example of a capitalized token in the data which is neither a named entity nor at the start of a sentence. What could be the reason the token is capitalized (one sentence)?

## Term Frequencies

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(min_df=0.01, 
                        max_df=0.9,  
                        max_features=1000,
                        stop_words='english',
                        use_idf=True, # the new piece
                        ngram_range=(1,2))

from wordcloud import WordCloud
import matplotlib.pyplot as plt

##TODO using the whole sample, produce a world cloud with bigrams for label == business using tfidf frequencies

## Supervised Feature Selection

In [ ]:
from sklearn.feature_selection import SelectKBest, f_classif, chi2

##TODO compute the number of words per document (excluding stopwords)
##TODO get the most predictive features of the number of words per document using first f_class and then chi2

Are the results different? What could be a reason for this? 

## Huggingface Tokenizers

In [ ]:
# # we use distilbert tokenizer
# !pip install transformers
from transformers import DistilBertTokenizerFast

# let's instantiate a tokenizer
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

##TODO tokenize the sentences in the sampled dataframe (dfs) using the DisilBertTokenizer
##TODO what is the type/token ratio from this tokenizer (number_of_unqiue_token_types/number_of_tokens)?
##TODO what is the amount of subword tokens returned by the huggingface tokenizer? hint: each subword token starts with "#"



# Parsing

In [ ]:
import pandas as pd
import nltk
df = pd.read_csv('train.csv')

df.columns = ["label", "title", "lead"]
label_map = {1:"world", 2:"sport", 3:"business", 4:"sci/tech"}
def replace_label(x):
	return label_map[x]
df["label"] = df["label"].apply(replace_label) 
df["text"] = df["title"] + " " + df["lead"]
df = df.sample(n=10000) # # only use 10K datapoints
df.head()

,label,title,lead,text
110581,sport,UEFA to probe Valencia-Werder Bremen incidents,UEFA will be launching disciplinary proceeding...,UEFA to probe Valencia-Werder Bremen incidents...
15925,sci/tech,New iMac tries to play it cool,Hot G5 chip requires some serious effort to av...,New iMac tries to play it cool Hot G5 chip req...
46720,business,Ford Reports Disappointing U.S. Sales,Ford's car and truck business sales fell nearl...,Ford Reports Disappointing U.S. Sales Ford's c...
57350,sci/tech,Microsoft Upgrades Windows XP Media Center (Ne...,NewsFactor - Bill Gates is about to announce a...,Microsoft Upgrades Windows XP Media Center (Ne...
90601,sport,Rams make statement they #39;re team to beat i...,ST. LOUIS -- When St. Louis coach Mike Martz l...,Rams make statement they #39;re team to beat i...


In [ ]:
import spacy
nlp = spacy.load('en_core_web_sm')
#TODO preprocess the corpus using spacy


### Information Extraction

In [ ]:
def extract_subject_verb_pairs(sent):
    subjs = [w for w in sent if w.dep_ == "nsubj"]
    pairs = [(w.lemma_.lower(), w.head.lemma_.lower()) for w in subjs]
    return pairs
##TODO extract the subject-verbs pairs and print the result for the second document

from collections import Counter
counter = Counter()

##TODO create a list ranking the most common pairs and print the first 10 items

In [ ]:
##TODO do the same for verbs-object pairs ('dobj')
##TODO create a list ranking the most common pairs and print the first 10 items

In [ ]:
##TODO do the same for adjectives-nouns pairs ('amod')
##TODO create a list ranking the most common pairs and print the first 10 items

### Exploring cross label dependencies

In [ ]:
##TODO extract all the subject-verbs and verbs-object pairs for the verb "rise"

In [ ]:
##TODO for each label create a list ranking the most common subject-verbs pairs and one for the most common verbs-object pairs
##TODO print the 10 most common pairs for each of the two lists for the labels "world" and "sci/tech"